<a href="https://colab.research.google.com/github/Tharun-C0/tharun-pep/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers torch accelerate gradio sympy


In [2]:
import torch
import re
import gradio as gr
import sympy as sp

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [3]:
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [4]:
def is_simple_math(expr):
    return bool(re.fullmatch(r"[0-9+\-*/(). ]+", expr))


In [5]:
def solve_math_exact(expr):
    try:
        result = sp.sympify(expr, evaluate=True)
        return result
    except:
        return None


In [6]:
def explain_math(expr, result):
    prompt = f"""
    Explain how to solve the following math problem step by step.
    Do NOT calculate the final answer.
    The final answer is already provided.

    Problem: {expr}
    Final Answer: {result}

    Explanation:
    """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=160,
        temperature=0.2,
        do_sample=False
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [7]:
def is_definition_question(text):
    keywords = [
        "define", "definition", "what is", "explain",
        "meaning of", "describe", "tell me about"
    ]
    return any(k in text.lower() for k in keywords)


In [8]:
conversation_history = []

def chatbot_response(user_input):


    if is_simple_math(user_input):
        exact_result = solve_math_exact(user_input)

        if exact_result is not None:
            explanation = explain_math(user_input, exact_result)

            return (
                f"🧮 **Math Solution**\n\n"
                f"Expression: {user_input}\n"
                f"Final Answer: {exact_result}\n\n"
                f"Explanation:\n{explanation}"
            )


    if is_definition_question(user_input):
        prompt = f"""
        You are an expert computer science professor.
        Give a clear, accurate, and well-structured explanation.
        Use simple language and examples if needed.

        Question: {user_input}
        Answer:
        """
    else:

        context = " ".join(conversation_history[-4:])
        prompt = f"""
        You are a friendly and intelligent AI assistant.
        Maintain a natural conversation.

        Conversation history:
        {context}

        User: {user_input}
        Assistant:
        """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.3,
        do_sample=False
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    conversation_history.append(f"User: {user_input}")
    conversation_history.append(f"Assistant: {response}")

    return response


In [9]:
def chat_ui(user_message, history):
    bot_reply = chatbot_response(user_message)
    history.append((user_message, bot_reply))
    return history, ""


In [10]:
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Level-5 Intelligent NLP + Math Chatbot")
    gr.Markdown("Handles conversation, definitions, and **accurate step-by-step math reasoning**")

    chatbot = gr.Chatbot(height=450)
    msg = gr.Textbox(placeholder="Ask anything (math, concepts, chat)...")
    clear = gr.Button("Clear Chat")

    msg.submit(chat_ui, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: [], None, chatbot)

demo.launch(share=False)


/tmp/ipython-input-341645437.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=450)
/tmp/ipython-input-341645437.py:5: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=450)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>